# Control-Flow

**Download this notebook - {nb-download}`control-flow.ipynb`**

In [1]:
from guppylang import guppy
from guppylang.std.builtins import result
from guppylang.std.quantum import measure, qubit, discard, h, tdg, cx, t, z

Implement an $V_3 = R_z(-2\arctan(2)) = (I + 2iZ)/\sqrt{5}$ gate using the repeat-until-success scheme from https://arxiv.org/abs/1311.1074.

In [2]:
@guppy
def repeat_until_success(q: qubit) -> None:
    attempts = 0
    while True:
        attempts += 1

        # Prepare ancillas
        a, b = qubit(), qubit()
        h(a)
        h(b)

        tdg(a)
        cx(b, a)
        t(a)
        h(a)
        if measure(a):
            # First part failed, try again
            discard(b)
            continue

        t(q)
        z(q)
        cx(q, b)
        t(b)
        h(b)
        if measure(b):
            # Second part failed, apply correction
            # and try again
            z(q)
            continue

        result("attempts", attempts)
        break

In [3]:
@guppy
def main() -> None:
    q = qubit()
    repeat_until_success(q)
    measure(q)

In [4]:
from selene_sim import build, Quest
from hugr.qsystem.result import QsysResult

runner = build(guppy.compile(main))

In [5]:
shots = QsysResult(runner.run_shots(Quest(random_seed=0), n_qubits=3, n_shots=100))

sum(int(shot.as_dict()["attempts"]) for shot in shots.results) / len(shots.results)

1.49